<img src="data/images/lecture-notebook-header.png" />

# Association Rule Mining (ARM)

Association Rule Mining (ARM) is a data mining technique that aims to discover interesting relationships or associations among items in large datasets. It focuses on finding patterns and dependencies between items to gain insights into various domains, including customer behavior, market basket analysis, web usage mining, and more.

In ARM, the primary objective is to identify frequent itemsets and generate association rules based on their occurrences. An itemset refers to a collection of items that appear together in a transaction or record. Association rules represent the relationships between different sets of items and are typically expressed in the form of "if-then" statements. For example, "If a customer buys product X and product Y, then they are likely to buy product Z."

Here are some common application use cases of association rule mining:

* **Market Basket Analysis:** One of the most well-known applications, market basket analysis examines customer purchase patterns to identify associations between products. By analyzing transaction data from retail stores or e-commerce platforms, association rule mining can reveal which items are frequently purchased together. This information is valuable for optimizing product placement, cross-selling, and designing targeted marketing campaigns.

* **Recommender Systems:** ARM is used in recommender systems to suggest related items or products to users. By analyzing the purchase history or behavior of users, the algorithm can identify associations and make personalized recommendations. For example, based on the items a user has already purchased, the system can recommend complementary products or items frequently bought together by similar users.

* **Customer Behavior Analysis:** ARM helps analyze customer behavior and preferences. By examining transactional data, website clickstreams, or user activity logs, associations can be discovered between the products or services customers interact with. This information aids in understanding customer preferences, segmenting customers, and developing targeted marketing strategies.

* **Fraud Detection:** ARM can be utilized to detect fraudulent activities or anomalies in large datasets. By analyzing patterns and associations in transactional data, the algorithm can identify unusual behaviors or deviations from normal patterns, flagging potential fraudulent transactions.

* **Bioinformatics:** In bioinformatics, ARM is used to uncover relationships between genes, proteins, and diseases. By analyzing biological data, researchers can identify associations between specific genetic variants, diseases, or other molecular factors, leading to advancements in personalized medicine and drug discovery.

These are just a few examples of how ARM is applied in various domains to discover meaningful associations and gain insights from large datasets. In this notebook, we consider the most popular use case: Market Basket Analysis.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Import Required Packages

For this notebook, you need to install the Python package [`efficient-apriori`](https://pypi.org/project/efficient-apriori/)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from efficient_apriori import apriori

---

## Visualizing the Complexity

### Brute Force Approach

We have seen in the lecture that given $d$ unique items in a dataset that there are $3^d - 2^{d+1} + 1$ possible association rules. To get a better sense how fast this number of rules increases with $d$, let's plot the corresponding graph. Feel free to change the value of `d_max` to increase the number of unique items $d$ and see the effects on the results -- well for `d_max > 38` the plot fails as the number of rules exceeds the range of the data type.

In [ ]:
d_max = 10
d = np.arange(2, d_max+1, 1)
y = np.power(3, d) - 2*np.power(2, d) + 1

plt.figure()
plt.xlabel('Number of unique items $\it{d}$', fontsize=16)
plt.ylabel('Number of Association Rules', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tight_layout()
plt.plot(d, y)
plt.grid(True)
plt.show()

Even for just 10 unique items, there are already 57k+ possible association rules. Now recall that the average supermarket has 10k-30k different items.

Of course, we can provide some very basic upper bound for the complexity by looking at the data. If we assume that the largest transaction contains $w$ items, and we have $N$ transactions with all transactions having different items, the number of possible rules is in $O(N\cdot (3^w - 2^{w+1} + 1))$. While $N$ can be a big number, $w$ is generally much smaller than $d$.

The plot below assumes a dataset of $N=1000$ transactions, with the largest transaction having $w\_max=5$ items. Keeping $N$ and $w\_max$ fixed, the upper bound $O(N\cdot (3^w - 2^{w+1} + 1))$ is also fixed, and you can see when the increasing values for $d$ of the naive approach yield the worse outcome.

In [ ]:
d_max = 12
w_max = 5
N = 1000

d = np.arange(2, d_max+1, 1)
w = np.array([w_max]*len(d))
y_d = np.power(3, d) - 2*np.power(2, d) + 1
y_w = N * np.power(3, w) - 2*np.power(2, w) + 1

plt.figure()
plt.xlabel('Number of unique items $\it{d}$', fontsize=16)
plt.ylabel('Number of Association Rules', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tight_layout()
plt.plot(d, y_d)
plt.plot(d, y_w)
plt.grid(True)
plt.show()

The simple take-away message here is that any kind of brute-force approach for Association Rule Mining is not applicable and/or practical for an real-world setting with a meaningful number of unique items (e.g., unique products sold in a supermarkt let alone in an online store with virtually no limitations).

---

## Frequent Itsemset Generation (Naive Approach)

We have seen in the lecture that we can decouple the calculation of support and confidence which allowed us to split the task of finding association rules into two parts:

* **(1) Frequent Itemset Generation:** Identify all frequent itemsets -- that is, itemsets with a support greater or equal to `minsup`. The observation was to only frequent itemset allow to derive rules with sufficient support
    
* **(2) Association Rules Generation:** Given all frequent itemsets, generate all possible rules and check if their confidence is above `minconf`.

Step (2) is arguably the less problematic one as the information required to calculate the confidence of a rule has already been generated during (1). We therefore focused on Frequent Itemset Generation. Again, the naive approach is to check all possible itemsets if their support is above `minsup`. Given $d$ unique items, we have seen that the number of possible itemsets is $2^d -1$. Again, we can plot this:

In [ ]:
d_max = 10
d = np.arange(2, d_max+1, 1)
y = np.power(2, d) - 1

plt.figure()
plt.xlabel('Number of unique items $\it{d}$', fontsize=16)
plt.ylabel('Number of Association Rules', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tight_layout()
plt.plot(d, y)
plt.grid(True)
plt.show()

While the number of possible itemsets grows slower than the number of possible association rules, it is still an exponential growth and hence not applicable for any practical scenario with $d$ being a realistic number in the 10,000 and above. This brought us to the Apriori Algorithm.

---

## Apriori Algorithm

The Apriori algorithm is one of the most well-known and widely used algorithms for association rule mining. It is designed to discover frequent itemsets and generate association rules from large datasets. The algorithm efficiently identifies itemsets that meet minimum support thresholds by utilizing the concept of the "Apriori property."

The Apriori Algorithm for ARM utilizes the fact the support of itemsets adheres to the following simple principle: If the an itemset `{a, b, c}` is infrequent (i.e., its support is below `minsup`), then all supersets, e.g., `{a, b, c, d}` are also infrequent and do not need to be checked. The basic intuition behind the Apriori Algorithm is to start with the smallest itemsets (`{a}`, `{b}`, `{b}`, ...) and continuously consider only those larger itemsets that potentially are frequent itemsets. We detailed this step-by-step process in the lecture with a complete example. Here's an overview of the Apriori algorithm:

* Step 1: Determine the minimum support: The user sets a minimum support threshold, indicating the minimum frequency or occurrence required for an itemset to be considered "frequent" in the dataset.

* Step 2: Generate frequent 1-itemsets: Scan the dataset to count the occurrences of each item. Items that meet the minimum support threshold are considered frequent 1-itemsets.

* Step 3: Generate frequent k-itemsets: Using the frequent 1-itemsets, generate candidate itemsets of length k (k > 1). To generate candidate k-itemsets, the Apriori property is employed, which states that any subset of a frequent k-itemset must also be frequent. Pruning techniques are applied to eliminate infrequent itemsets efficiently.

* Step 4: Prune infrequent itemsets: Scan the dataset to count the occurrences of the candidate k-itemsets. Itemsets that do not meet the minimum support threshold are pruned and not considered frequent. Only the remaining frequent k-itemsets are retained.

* Step 5: Repeat Steps 3 and 4 until no more frequent itemsets can be generated.

* Step 6: Generate association rules: From the frequent itemsets, generate association rules by considering different subsets of items. Calculate the confidence of each rule and discard rules that do not meet the minimum confidence threshold.

The Apriori algorithm progressively generates increasingly large itemsets by iteratively scanning the dataset and pruning infrequent itemsets. This approach reduces the search space and improves efficiency compared to exhaustive enumeration.

The Apriori algorithm is widely used due to its simplicity and effectiveness in identifying frequent itemsets and generating association rules. However, one limitation is that it may encounter performance issues when dealing with large datasets or datasets with a high number of unique items. Various optimization techniques, such as pruning strategies, transaction reduction, and vertical data format, can be applied to enhance the efficiency of the algorithm in such cases.

Although the basic idea behind the Apriori Algorithm is arguably rather intuitive, implementing it from scratch to run efficiently is not trivial. In this notebook, we therefore use the existing Python package [`efficient-apriori`](https://pypi.org/project/efficient-apriori/).

### Toy Dataset

The following dataset with 5 transactions and 6 different items is directly taken from the lecture slides.

In [ ]:
transactions = [
    ('bread', 'yogurt'),
    ('bread', 'milk', 'cereal', 'eggs'),
    ('yogurt', 'milk', 'cereal', 'cheese'),
    ('bread', 'yogurt', 'milk', 'cereal'),
    ('bread', 'yogurt', 'milk', 'cheese')
]

### Running the Apriori Algorithm

In [ ]:
itemsets, rules = apriori(transactions, min_support=0.4, min_confidence=0.8, max_length=4)

`itemsets` contains all **frequent itemsets** -- that is, all k-itemsets from 1 to `max_length` (including their support counts) with a support greater or equal than `min_support`. Try increasing `min_support` and see how the number of frequent itemsets decreases.

In [ ]:
for k, k_itemsets in itemsets.items():
    print('All {}-itemsets:'. format(k))
    for k_itemset, support_count in k_itemsets.items():
        print('\t{}-itemset {} -- support count: {}'.format(k, k_itemset, support_count))

`rules` contains all **association rules** derived from the frequent itemsets -- which mean they also have sufficient support -- that have a confidence greater equal than `min_confidence`.

In [ ]:
for rule in rules:
    print(rule)

You can access the individual components and information of a rule as follows:

In [ ]:
for rule in rules:
    left_hand_side = rule.lhs    # or antecedent
    right_hand_side = rule.rhs   # or consequent
    support = rule.support
    confidence = rule.confidence
    lift = rule.lift
    conviction = rule.conviction
    print('{} => {} -- lift: {}'.format(left_hand_side, right_hand_side, lift))

You can easily filter rules if needed. For example, below we are only interested in rules where the antecedent contains 2 items and the consequent 1 item.

In [ ]:
rules_filtered = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)

for rule in rules_filtered:
    print(rule)

In practice, you're often interested in the "best" rules which implies a ranking. A common sorting criteria to find the best rules is the lift.

In [ ]:
rules_sorted_by_lift = sorted(rules, key=lambda rule: rule.lift, reverse=True)

for rule in rules_sorted_by_lift:
    print(rule)

In the previous example, we sorted the initial outcome of the Apriori Algorithm. Of course, you can first filter the rules as needed and apply sorting/ranking on the filtered results.

### Performance on Toy Dataset

Our toy dataset has only 5 transactions and 6 unique items. So w.r.t. to performance, you would not see any difference between the Apriori algorithm and the naive approach. However, once we work with a real-world dataset with several 100k transactions and several 1k unique items, the naive approach is simply out of the question. Note that even the Apriori algorithm is likely to take quite some time depending on your available computing power but also on the choices of values for `minsup` and `minconf`. It is therefore often a good idea to first start with large(r) values for `minsup` to get some first results more quickly. If the set of association rules is empty or too small, you can then start lowering `minsup` but also `minconf` step by step.

---

## Real-World Dataset

In the following, we use the [Online Retail II](https://archive.ics.uci.edu/ml/machine-learning-databases/00502/) data set that contains all the transactions occurring for a UK-based and registered, non-store online retail. The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

**Important:** We already performed some data cleaning for you and kept only transaction with not more than 10 items just so the execution times will be fast enough to try different parameters and see their effects on the results.

### Loading the Dataset (Sample)

The dataset is provided as a comma-separated file, so we can use `pandas` to load it into a DataFrame for further processing

In [ ]:
df_retail = pd.read_csv('data/datasets/online-retail/online-retail-sample.csv')

df_retail.head(10)

Let's check the basic information about the dataset.

In [ ]:
num_records, num_attributes = df_retail.shape

print('There are {} records with {} attributes'.format(num_records, num_attributes))

Since association rules will only have stock codes as antecedents and consequents, they are not easy to interpret. To easily map stock codes to descriptions, we can simply generate a dictionary for later use. This might take a couple of seconds.


In [ ]:
%%time
code2desc = { str(row['StockCode']):str(row['Description']).strip() for  idx, row in df_retail.iterrows() }

Given a stock code, we can now look up the description -- for example:

In [ ]:
stock_code = '22064'  # Stock codes a strings as some of them contains letters

print('The description for stock code {} is "{}"'.format(stock_code, code2desc[stock_code]))

### Converting the Data

The `efficient-apriori` package assumes as input a list of transactions; see `transactions_demo`. However, right now we have all transactions in table-like format. We therefore need to transform the data. Note that using the `StockCode` to represent an item will suffice; we don't need the whole description here. It also saves memory as stock codes are typically much shorter than descriptions.

The conversion from the DataFrame into a list of transactions can be done using `pandas` by querying the DataFrame. More specifically, we can apply grouping and aggregation to get the desired result; check out the Pandas notebook for more details. The code cell below performs the required code for the data conversion.

In [ ]:
transactions_retail = df_retail.groupby(['Invoice']).agg({'StockCode': tuple})['StockCode'].to_list()

Let's have a look at an example transaction.

In [ ]:
transactions_retail[0]

Now our dataset is in the right format to serve as input for the implementation of the Apriori algorithm of the [`efficient-apriori`](https://pypi.org/project/efficient-apriori/) package.

### Running the Apriori Algorithm

Run the Apriori algorithm of the [`efficient-apriori`](https://pypi.org/project/efficient-apriori/) package with different parameters to see the effects in the results as well as on the runtime. By default, we use a `minsup` value of 0.005 to ensure a fast runtime.

In [ ]:
%%time

itemsets, rules = apriori(transactions_retail, min_support=0.005, min_confidence=0.2, max_length=4)

rules_filtered = rules

## Note: you would need to lower the supprt quite a bit to get rules with more than
## 1 item on the left or right-hand side for this sample dataset
#rules_filtered = filter(lambda rule: len(rule.lhs) > 1 or len(rule.rhs) > 1, rules)

top10_rules = sorted(rules_filtered, key=lambda rule: rule.lift, reverse=True)[0:10]

In [ ]:
for rule in top10_rules:
    # We can convert the StockCode back to the description to make the rules more readable
    # Note the some descriptions are NA (although the StockCode is valid)
    antecedent = [ code2desc[c] for c in rule.lhs]
    consequent = [ code2desc[c] for c in rule.rhs]
    print('{} => {} -- lift: {}'.format(antecedent, consequent, rule.lift))

As you have noticed, for real-world datasets, a common value for `minsup` is much lower as you have seen in the lecture and in the example using the toy dataset. This shouldn't be surprising as support reflects how many transaction contain an itemset. So a support of 1% (0.01) requires that frequent itemsets have to be indeed in 1% of all transactions. In real-world datasets, this indicates already very popular item.

You will also have noticed, that the results ranked by lift comes in pairs, e.g., the first two rules are the same just with the antecedent and the consequent flipped. This is to be expected since lift -- compared to, e.g., confidence -- is a symmetric metric.

---

## Summary

Association Rule Mining (ARM) is an important and widely applicable data mining task. It is a data mining technique used to uncover interesting relationships or associations between items in large datasets. It allows businesses to discover patterns, dependencies, and correlations, providing valuable insights into customer behavior, market trends, and more. By analyzing transactional or relational databases, association rule mining helps organizations optimize marketing strategies, improve decision-making, and enhance customer satisfaction.

One of the primary advantages of association rule mining is its ability to identify hidden patterns and associations that may not be immediately apparent. It enables businesses to discover meaningful relationships between items, which can be leveraged for targeted marketing campaigns, cross-selling, and upselling. Association rules also offer interpretability, as they can be expressed in a simple "if-then" format, making it easier to understand and act upon the discovered patterns.

However, association rule mining has certain limitations. Firstly, it may produce a large number of rules, including trivial or spurious associations, which can be overwhelming and misleading. Careful analysis and filtering are necessary to extract meaningful insights. Secondly, association rules are based on correlations rather than causations. While the presence of an association indicates a relationship, it does not necessarily imply a cause-and-effect relationship. Caution should be exercised in interpreting and making decisions based solely on association rules.

Lastly, while the notion of a "useful rule" based on support, confidence, lift, etc. is rather intuitive, association rule mining can be computationally expensive, especially for datasets with a high number of items or large transactional databases. The Apriori algorithm, commonly used for association rule mining, may encounter scalability issues. However, there are optimization techniques available, such as pruning strategies and parallelization, to improve efficiency and handle larger datasets.

There are many ways to extend basic ARM covered here. For example, so far we considered transactions as simple sets of items. In practice, transactions may include further information that can be utilized to extract association rules, for example:

* Attributes of items (e.g., quantity and price of products)
* Sequence of items (e.g., order when products have be added to the cart)
* Categories of items (e.g., "milk" and "yogurt" are both "dairy" products)
* User information (e.g., associating multiple transactions to the same user)

In summary, association rule mining is a valuable data mining technique that uncovers associations and patterns between items. It provides actionable insights for businesses but requires careful interpretation and filtering of results. While offering interpretability and the ability to discover hidden relationships, it is essential to be aware of potential limitations, such as the risk of spurious associations and the need for computational efficiency.